# Загрузка данных

In [ ]:
f = open('Text.txt')
full_text=f.read()

In [ ]:
len(full_text)

2904

# Кодирование каждого уникального символа 8-битной последовательностью

In [ ]:
def encoding(text):
  dictionary={'':'00000000'}
  code=1
  for i in range(len(text)):
    if text[i] not in dictionary:
      temp=str(bin(code)[2:])
      while len(temp)<8:
        temp='0'+str(temp)
      dictionary[text[i]] =temp
      code+=1
  return dictionary

In [ ]:
word_dict=encoding(full_text)

# Составление информационных слов для кода Хемминга

In [ ]:
def codewords(text,dictionary):
  bin_text=''
  words=[]
  for i in text:
    bin_text+=dictionary[i]
  for i in range(0,len(bin_text),103):
    words.append(bin_text[i:i+103])
    while len(words[-1])<103:
      words[-1]+='0'
  return words

In [ ]:
code_words=codewords(full_text,word_dict)

# CRC-64, вычисление первых контрольных сумм для каждого слова


In [ ]:
!pip install crc64iso

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from crc64iso.crc64iso import crc64
checksum1=[crc64(i) for i in code_words]

# Код Хемминга, добавление контрольных бит в информационное слово

In [ ]:
def hamming_code(words):
  hamming_code=[]
  for i in range(len(words)):
    hamming_code.append(words[i])
    for j in range(7):
      hamming_code[i]=hamming_code[i][:2**j-1]+str(0)+hamming_code[i][2**j-1:]
    for j in range(7):
      start= 2**j-1+1
      stop=2*2**j-1
      line_sum=0
      while start<=(len(str(hamming_code[i]))):
        arr=[int(i) for i in hamming_code[i][start:stop]]
        line_sum+=sum(arr)
        start=stop+2**j
        stop=start+2**j
      hamming_code[i]=hamming_code[i][:2**j-1]+str(line_sum%2)+hamming_code[i][2**j:]
  return hamming_code

In [ ]:
hamming_words=hamming_code(code_words)
print('Длина информационного слова с учетом контрольных бит:',len(hamming_words[0]))
print('Количество информационных слов:',len(hamming_words))

Длина информационного слова с учетом контрольных бит: 110
Количество информационных слов: 226


# Генерация случайных ошибок

Генерация рандомных ошибок для каждого слова

In [ ]:
from random import randint

random_numbers=[]
for i in range(len(hamming_words)):
  random_numbers.append(randint(0, len(hamming_words[i])-1))

Генерация рандомных номеров преложений для добавления нескольких ошибок в предложения

In [ ]:
from random import randint

random_sentence=[]
for i in range(40):
  random_sentence.append(randint(len(hamming_words)-20, len(hamming_words)-1))

In [ ]:
import pandas as pd
pd.Series(random_sentence).value_counts()

209    5
210    3
220    3
207    3
212    3
215    3
206    3
214    2
219    2
213    2
216    2
221    2
224    2
225    2
211    1
217    1
222    1
dtype: int64

# Внесение ошибок

Сообщение без ошибок

In [ ]:
mistakes_0=hamming_words.copy()

Внесение ошибки в каждое слово

In [ ]:
mistakes_1=hamming_words.copy()
for i in range(len(hamming_words)):
  mistakes_1[i]=mistakes_1[i][:random_numbers[i]]+str(abs(int(mistakes_1[i][random_numbers[i]])-1))+mistakes_1[i][random_numbers[i]+1:]

Внесение нескольких ошибок 

In [ ]:
mistakes_few=hamming_words.copy()
for i in range(len(random_sentence)):
  mistakes_few[random_sentence[i]]=mistakes_few[random_sentence[i]][:random_numbers[i]]+str(abs(int(mistakes_few[random_sentence[i]][random_numbers[i]])-1))+mistakes_few[random_sentence[i]][random_numbers[i]+1:]

Выявление ошибок с помощью кода Хемминга

In [ ]:
def err_sent_detection(mistakes):
  mistakes_checker=[]

  for i in range(len(mistakes)):
    mistakes_checker.append(mistakes[i])
    for j in range(7):
      start= 2**j-1+1
      stop=2*2**j-1
      line_sum=0
      while start<=(len(str(mistakes_checker[i]))):
        arr=[int(i) for i in mistakes_checker[i][start:stop]]
        line_sum+=sum(arr)
        start=stop+2**j
        stop=start+2**j
      mistakes_checker[i]=mistakes_checker[i][:2**j-1]+str(line_sum%2)+mistakes_checker[i][2**j:]
  numb_sent=[]
  for i in range(len(mistakes_checker)):
    for j  in range(len(mistakes_checker[i])):
      if(mistakes_checker[i][j]!=mistakes[i][j]):
        # print("There is a mistake, sentence number ",i)
        numb_sent.append(i)
        break
  return numb_sent

In [ ]:
numb_sent_0=err_sent_detection(mistakes_0)
numb_sent_1=err_sent_detection(mistakes_1)
numb_sent_few=err_sent_detection(mistakes_few)

# Поиск ошибок

In [ ]:
def mistakes_detection(numb_sent,mistakes):
  mistakes_checker=[]
  mis=[]
  for i in range(len(numb_sent)):
    mistakes_checker.append(mistakes[numb_sent[i]])
    mis.append([])
    for j in range(7):
      start= 2**j-1
      stop=2*2**j-1
      line_sum=0
      while start<(len(str(mistakes_checker[i]))):
        arr=[int(i) for i in mistakes_checker[i][start:stop]]
        line_sum+=sum(arr)
        start=stop+2**j
        stop=start+2**j
      mis[i].append(line_sum%2)
  results=[]
  for i in range(len(mis)):
    result=0
    for j in reversed(range(7)):
      result+=mis[i][j]*2**j
    results.append(result-1)
  return results
      

In [ ]:
mist_pos_0=mistakes_detection(numb_sent_0,mistakes_0)
mist_pos_1=mistakes_detection(numb_sent_1,mistakes_1)
mist_pos_few=mistakes_detection(numb_sent_few,mistakes_few)

# Исправление ошибок

In [ ]:
def mistakes_fixing(mist_pos,numb_sent, mistakes):
  words=mistakes.copy()
  for i in range(len(mist_pos)):
    try:
      words[numb_sent[i]]=words[numb_sent[i]][:mist_pos[i]]+str(abs(int(words[numb_sent[i]][mist_pos[i]])-1))+words[numb_sent[i]][mist_pos[i]+1:]
    except IndexError: continue
  return words


In [ ]:
words_0 = mistakes_fixing(mist_pos_0,numb_sent_0, mistakes_0)
words_1 = mistakes_fixing(mist_pos_1,numb_sent_1, mistakes_1)
words_few = mistakes_fixing(mist_pos_few,numb_sent_few, mistakes_few)

# Удаление контрольных бит

In [ ]:
def decoding(words):
  decoder=[]
  for i in range(len(words)):
    decoder.append(words[i])
    for j in reversed(range(7)):
      decoder[i]=decoder[i][:2**j-1]+decoder[i][2**j:]
  return decoder

In [ ]:
dec_words_0=decoding(words_0)
dec_words_1=decoding(words_1)
dec_words_few=decoding(words_few)

# Вычисление новых контрольных сумм

In [ ]:
from crc64iso.crc64iso import crc64
checksum2=[crc64(i) for i in dec_words_0]

In [ ]:
checksum1==checksum2

True

In [ ]:
from crc64iso.crc64iso import crc64
checksum2=[crc64(i) for i in dec_words_1]

In [ ]:
checksum1==checksum2

True

In [ ]:
from crc64iso.crc64iso import crc64
checksum2=[crc64(i) for i in dec_words_few]

In [ ]:
checksum1==checksum2

False

In [ ]:
numb_sent_few==sorted(list(set(random_sentence)))

True

In [ ]:
for i in numb_sent_few:
  print(i,checksum1[i]==checksum2[i])

206 False
207 False
209 False
210 False
211 True
212 False
213 False
214 False
215 False
216 False
217 True
219 False
220 False
221 False
222 True
224 False
225 False


In [ ]:
pd.Series(random_sentence).value_counts()

209    5
210    3
220    3
207    3
212    3
215    3
206    3
214    2
219    2
213    2
216    2
221    2
224    2
225    2
211    1
217    1
222    1
dtype: int64

Вывод

1) сообщения без ошибок были верно пререданы и контрольные суммы всех сообщений совпадают с исходными;

2) все одиночные ошибки были найдены и исправлены в каждом предложении;

3) были выявлены все предложения с ошибками, но были исправлены только одиночные ошибки.

# Декодинг

In [ ]:
def get_key(d, value):
  try:
    for k, v in d.items():
      if v == value:
        return k
  except TypeError:
    print(d[0])
    return d[0]


In [ ]:
def bin2words(dec_words):
  full_bin=''
  for i in range(len(dec_words)):
    full_bin+=dec_words[i]
  final_text=''
  for j in range(0,len(full_bin)-7,8):
      try:
        final_text+=get_key(word_dict,full_bin[j:j+8])
      except TypeError:
        final_text+=''
  return final_text

In [ ]:
text_0=bin2words(dec_words_0)
text_1=bin2words(dec_words_1)
text_few=bin2words(dec_words_few)

In [ ]:
text_0==full_text

True

In [ ]:
text_1==full_text

True

In [ ]:
text_few==full_text

False

In [ ]:
text_few

'Допустим, у нас есть сообщение «habr», которое необходимо передать без ошибок. Для этого сначала нужно наше сообщение закодировать при помощи Кода Хэмминга. Нам необходимо представить его в бинарном виде. На этом этапе стоит определиться с, так называемой, длиной информационного слова, то есть длиной строки из нулей и единиц, которые мы будем кодировать. Допустим, у нас длина слова будет равна 16. Таким образом, нам необходимо разделить наше исходное сообщение («habr») на блоки по 16 бит, которые мы будем потом кодировать отдельно друг от друга. Так как один символ занимает в памяти 8 бит, то в одно кодируемое слово помещается ровно два ASCII символа. Итак, мы получили две бинарные строки по 16 бит. После этого процесс кодирования распараллеливается, и две части сообщения («ha» и «br») кодируются независимо друг от друга. Рассмотрим, как это делается на примере первой части. Прежде всего, необходимо вставить контрольные биты. Они вставляются в строго определённых местах — это позиции 

In [ ]:
full_text

'Допустим, у нас есть сообщение «habr», которое необходимо передать без ошибок. Для этого сначала нужно наше сообщение закодировать при помощи Кода Хэмминга. Нам необходимо представить его в бинарном виде. На этом этапе стоит определиться с, так называемой, длиной информационного слова, то есть длиной строки из нулей и единиц, которые мы будем кодировать. Допустим, у нас длина слова будет равна 16. Таким образом, нам необходимо разделить наше исходное сообщение («habr») на блоки по 16 бит, которые мы будем потом кодировать отдельно друг от друга. Так как один символ занимает в памяти 8 бит, то в одно кодируемое слово помещается ровно два ASCII символа. Итак, мы получили две бинарные строки по 16 бит. После этого процесс кодирования распараллеливается, и две части сообщения («ha» и «br») кодируются независимо друг от друга. Рассмотрим, как это делается на примере первой части. Прежде всего, необходимо вставить контрольные биты. Они вставляются в строго определённых местах — это позиции 